# ShorelineMonitor: Change Rate

The ShorelineMonitor dataset provides Satellite-Derived Shorelines (SDS) extracted from annually
composited Landsat satellite imagery spanning the years 1984-2024. These shorelines offer a global
view of coastal change and shoreline dynamics, serving as a foundation for coastal
analytics, modeling and management. This notebook shows how to explore multi-decadal trends in shoreline change.

In [ ]:
import os

import dotenv
import fsspec
import geopandas as gpd
import hvplot.pandas
import pandas as pd
import pystac
import shapely
from dotenv import load_dotenv
from ipyleaflet import Map, basemaps

from coastpy.stac.utils import read_snapshot

load_dotenv()

# Configure cloud and Dask settings
sas_token = os.getenv("AZURE_STORAGE_SAS_TOKEN")
storage_options = {"account_name": "coclico", "sas_token": sas_token}

coclico_catalog = pystac.Catalog.from_file(
    "https://coclico.blob.core.windows.net/stac/v1/catalog.json"
)
collection = coclico_catalog.get_child("gctr")

In [ ]:
snapshot = read_snapshot(collection, storage_options=storage_options)
snapshot.head()

In [ ]:
snapshot.explore()

In [ ]:
from ipyleaflet import Map, basemaps

m = Map(basemap=basemaps.Esri.WorldImagery, scroll_wheel_zoom=True)
m.center = (43.32, -1.97)
m.zoom = 14
m.layout.height = "800px"
m

In [ ]:
from coastpy.geo.utils import get_region_of_interest_from_map

roi = get_region_of_interest_from_map(m, default_extent=(4.796, 53.108, 5.229, 53.272))

## Fetch data from the database

In [ ]:
import coastpy

db = coastpy.io.STACQueryEngine(
    stac_collection=collection,
    storage_backend="azure",
    # columns = ["geometry", "transect_id", "sds:change_rate"] ... # when you don't need all data
)

In [ ]:
from coastpy.utils.config import fetch_sas_token

sas_token = fetch_sas_token(sas_token)
df = db.get_data_within_bbox(west, south, east, north, sas_token=sas_token)
print(f"Shape: {df.shape}")
df.head()

In [ ]:
ac = gpd.GeoDataFrame(
    df[["transect_id", "sds:change_rate"]],
    geometry=gpd.GeoSeries.from_xy(df.lon, df.lat, crs=4326),
)

In [ ]:
import hvplot.pandas  
import colorcet as cc

def plot_shoreline_change_trend(ac_gdf):
    """
    Simple hvplot visualization for shoreline change trends.
    """
    # Calculate the 2.5th and 97.5th percentiles to reduce extreme outliers
    lower_quantile = ac_gdf["sds:change_rate"].quantile(0.025)
    upper_quantile = ac_gdf["sds:change_rate"].quantile(0.975)

    # Define symmetric color limits around zero for clear erosion/accretion visualization
    max_abs_range = max(abs(lower_quantile), abs(upper_quantile))
    clim = (-max_abs_range, max_abs_range)
    
    plot = ac_gdf.hvplot(
        geo=True,
        tiles="EsriImagery",  
        color="sds:change_rate",
        cmap=cc.CET_D3[::-1],  
        clim=clim,
        frame_width=800,
        frame_height=600,
        colorbar=True,
        title="Shoreline Change Trend (m/year)",
        tools=["hover"],
    )
    
    return plot

In [ ]:
plot_shoreline_change_trend(ac)